# Least Squares selection of the 30 features

In [1]:
# TO RUN IT: CHANGE MY PATH
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
# change path if necessary
import sys
my_path = r'/home/ilaria/Scrivania/Machine_Learning/Project_1/Project1_ML'
sys.path.insert(0,my_path + r'/code/COMMON')

# imports
import numpy as np 
import matplotlib.pyplot as plt

from proj1_helpers import predict_labels 
from implementations import *
from labels import idx_2labels
from costs import *
from optimize_hyperparams import *
from cross_validation import *
from step_wise import *
from extend_features import *

# Build Dataset 

In [2]:
from proj1_helpers import load_csv_data 

# load raw data
y_raw, input_data_raw, ids = load_csv_data(my_path + r'/data/train.csv', sub_sample=False)

In [3]:
from outliers import handle_outliers

# handle outliers
X0, y = handle_outliers(input_data_raw, y_raw, -999, 'mean')

-999 are replaced by the mean value of the feature


In [4]:
# set y in {0,1} instead of {-1,1}
y[np.where(y==-1)]=0

# Build features

In [76]:
# get feature names 
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
features_names = []
for i in range(len(names)):
    features_names.append(names[i])
    
all_candidates = X0
all_candidates.shape

(250000, 30)

In [5]:
features = []
for i in range(len(features_names)):
    features.append((i,features_names[i]))

NameError: name 'features_names' is not defined

In [7]:
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 1
X0, features = extend_features(input_data_raw, names, degree, log)
all_candidates = X0

---------------------------
Features have been set to the power(s): [1]
16 Features of the momentum have been added
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [8]:
all_candidates.shape

(250000, 50)

In [9]:
# STEPWISE WITH LEAST SQUARE
# parameters
model = dict()
model['method'] = 'ls'
model['loss'] = 'rmse'
#model['gamma'] = 1e-3
#model['max_iters'] = 5000
#model['threshold'] = 1e-4
#model['debug_mode'] = 1
#model['method_minimization'] = 'gd'

# R2 type
R2_method = 'McFadden' # or 'loss'

# estimate R2 error through cross validation (1 or 0)
cv = 0

best_R2adj, idx_features = stepwise(model, R2_method, all_candidates, features, y, cv)

cv: 0 L0: 0.761666237967
R2adj_chosen: 0.0320794565365
--------------------------------------------------------------------------------------------
Feature chosen:  DER_mass_transverse_met_lep_power_1 (index : 1 ) |  R2adj =  0.0320794565365
R2adj_chosen: 0.0411265530781
--------------------------------------------------------------------------------------------
Feature chosen:  PRI_tau_pt_power_1 (index : 13 ) |  R2adj =  0.0411265530781
R2adj_chosen: 0.0476859241161
--------------------------------------------------------------------------------------------
Feature chosen:  DER_deltaeta_jet_jet_power_1 (index : 4 ) |  R2adj =  0.0476859241161
R2adj_chosen: 0.0513540500361
--------------------------------------------------------------------------------------------
Feature chosen:  DER_met_phi_centrality_power_1 (index : 11 ) |  R2adj =  0.0513540500361
R2adj_chosen: 0.0541217005304
--------------------------------------------------------------------------------------------
Feature cho

In [10]:
# display selected features
results_r2_stepwise(best_R2adj[:len(best_R2adj)-1], idx_features)

step 1 : R2 adjusted = 0.0320794565365
step 2 : R2 adjusted = 0.0411265530781
step 3 : R2 adjusted = 0.0476859241161
step 4 : R2 adjusted = 0.0513540500361
step 5 : R2 adjusted = 0.0541217005304
step 6 : R2 adjusted = 0.0619509471832
step 7 : R2 adjusted = 0.0636698162768
step 8 : R2 adjusted = 0.0645875366324
step 9 : R2 adjusted = 0.065537927064
step 10 : R2 adjusted = 0.0663104102943
step 11 : R2 adjusted = 0.0670029136786
step 12 : R2 adjusted = 0.0676125537267
step 13 : R2 adjusted = 0.0685500923705
step 14 : R2 adjusted = 0.0701328057152
step 15 : R2 adjusted = 0.0708869018379
step 16 : R2 adjusted = 0.071204342374
step 17 : R2 adjusted = 0.0715130336785
step 18 : R2 adjusted = 0.0721154218984
step 19 : R2 adjusted = 0.0727543538223
step 20 : R2 adjusted = 0.0730407667631
step 21 : R2 adjusted = 0.0734366365681
step 22 : R2 adjusted = 0.0737501026612
step 23 : R2 adjusted = 0.0738255571317
step 24 : R2 adjusted = 0.0738659884399
step 25 : R2 adjusted = 0.0739037116954
step 26 : R

In [11]:
X = X0[:,idx_features]
X.shape

(250000, 30)

# Adding new features : logarithm + momentum (module and first component)

In [85]:
# adding the 4 logarithm of the masses
#index_log = [0,1,2,5]
#for i, indx in enumerate(index_log):
#    X = np.concatenate((X, np.log(1+np.abs(X0[:,indx])).reshape(len(X),1)), axis = 1)

#print(X.shape)

# adding momentum
#features_4 = X0[:,[13,16,23,26]]
#eta_angles = X0[:, [14,17,24,27]]
#phi_angles = X0[:,[15,18,25,28]]        

#P_mom_comp1 = features_4*np.cos(phi_angles)
#P_mom_mod = features_4*np.cosh(eta_angles)

#X = np.concatenate((X, P_mom_mod), axis = 1)
#print(X.shape)
#X = np.concatenate((X,P_mom_comp1), axis = 1)
#print(X.shape)

(250000, 21)


In [86]:
# standardize

#X, _,_ = standardize(X)

# Apply the logistic regression with parameter fixed

In [12]:
initial_w = np.ones(X.shape[1])
max_iters = 1000
gamma = 1e-4
method = 'gd'

w, loss = logistic_regression(y,X,initial_w,max_iters, gamma, method)

In [13]:
yc = predict_labels_log(w[-1],X)
yc.shape

(250000,)

In [14]:
pos = 0
neg = 0
for i in range (len(y)):
    if y[i] == yc[i]:
        pos += 1
    else:
        neg += 1
print(pos)
print(neg)

success_rate = pos/(pos+neg)
print(success_rate)

182497
67503
0.729988


# Load the datatest

In [15]:
y, test_data, ids = load_csv_data(r'/home/ilaria/Scrivania/Machine_Learning/Project_1/test.csv', sub_sample=False)

print("Data loaded! Shape: ")
print(np.shape(test_data))

Data loaded! Shape: 
(568238, 30)


In [16]:
input_data, Y = handle_outliers(test_data,y,-999,'mean')

-999 are replaced by the mean value of the feature


In [94]:
#X = input_data[:,idx_features]
#print(X.shape)

# adding the 4 logarithm of the masses
#index_log = [0,1,2,5]
#for i, indx in enumerate(index_log):
#    X = np.concatenate((X, np.log(1+np.abs(X[:,indx])).reshape(len(X),1)), axis = 1)

#print(X.shape)

# adding momentum
#features_4 = input_data[:,[13,16,23,26]]
#eta_angles = input_data[:, [14,17,24,27]]
#phi_angles = input_data[:,[15,18,25,28]]        

#P_mom_comp1 = features_4*np.cos(phi_angles)
#P_mom_mod = features_4*np.cosh(eta_angles)

#X = np.concatenate((X, P_mom_mod), axis = 1)
#print(X.shape)
#X = np.concatenate((X, P_mom_comp1), axis = 1)
#print(X.shape)

#X, _,_ = standardize(X)

(568238, 17)
(568238, 21)


In [23]:
names = list(np.genfromtxt(my_path + r'/data/train.csv', delimiter=",", dtype=str, max_rows = 1)[2:])
log = True
degree = 1
X0, features = extend_features(input_data, names, degree, log)

---------------------------
Features have been set to the power(s): [1]
16 Features of the momentum have been added
4 logarithmic features have been added.
Data have been standardized.
---------------------------


In [24]:
X = X0[:,idx_features] 
X.shape

(568238, 30)

In [25]:
y_pred = predict_labels_log(w[-1],X)

In [26]:
y_pred[np.where(y_pred==0)] = -1

In [27]:
create_csv_submission(ids, y_pred, "b_sub")